# Resolve did:btc1 from local Bitcoin node in regtest

In [1]:
import sys
import os

notebooks_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the Notebooks directory to the sys.path
sys.path.append(notebooks_path)

In [2]:
did_to_resolve = "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc"

# Signal Meta
# - id
# - updatePayload
# btc1SidecarData

sidecar_data = {
  "did": "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc",
  "signalsMetadata": {
    "5f8cd13f39fa509b1cdfdc7c6588b6cda99e82202e9498dff9f37dc99d4a1e10": {
      "updatePayload": {
        "@context": [
          "https://w3id.org/security/v2",
          "https://w3id.org/zcap/v1",
          "https://w3id.org/json-ld-patch/v1"
        ],
        "patch": [
          {
            "op": "add",
            "path": "/service/3",
            "value": {
              "id": "#linked-domain",
              "type": "LinkedDomains",
              "serviceEndpoint": "https://contact-me.com"
            }
          }
        ],
        "sourceHash": "9kSA9j3z2X3a26yAdJi6nwg31qyfaHMCU1u81ZrkHirM",
        "targetHash": "C45TsdfkLZh5zL6pFfRmK93X4EdHusbCDwvt8d7Xs3dP",
        "targetVersionId": 2,
        "proof": {
          "type": "DataIntegrityProof",
          "cryptosuite": "bip340-jcs-2025",
          "verificationMethod": "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc#initialKey",
          "proofPurpose": "capabilityInvocation",
          "capability": "urn:zcap:root:did%3Abtc1%3Aregtest%3Ak1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc",
          "capabilityAction": "Write",
          "@context": [
            "https://w3id.org/security/v2",
            "https://w3id.org/zcap/v1",
            "https://w3id.org/json-ld-patch/v1"
          ],
          "proofValue": "z3yfzVGdoDF4s8y4Bk8JeV9XuZw1nMeMtNW3x5brEm7DNtmWZkNBPbCLzUBJRpctBj9QJL1dydm94ZNsPxosPnkPP"
        }
      }
    }
  }
}

In [3]:

resolutionOptions = {"sidecarData": sidecar_data}

In [4]:
import os
ROOT_DIR = os.path.dirname(os.path.abspath('..'))

## Parse did:btc1 identifier

In [5]:
import re
match = re.search(r"^did:btc1:(?:(\d+):)?(?:(mainnet|signet|testnet|regtest):)?((k1|x1)[023456789acdefghjklmnpqrstuvwxyz]*)$", did_to_resolve)
groups = match.groups()
groups

(None,
 'regtest',
 'k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc',
 'k1')

In [6]:
groups[2]

'k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc'

In [7]:
# Folder path
folder_path = f"{ROOT_DIR}/TestVectors/{groups[2]}"

# Ensure the folder exists
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [8]:
import json

file_path = f"{folder_path}/did.txt"
with open(file_path, "w") as f:
    f.write(did_to_resolve)

In [9]:
import json

file_path = f"{folder_path}/resolutionOptions.json"
with open(file_path, "w") as f:
    json.dump(resolutionOptions, f, indent=2)

In [10]:
identifierComponents = {}

identifierComponents["version"] = groups[0] or 1
identifierComponents["network"] = groups[1] or "mainnet"




In [11]:
from libbtc1.bech32 import decode_bech32_identifier

bech32_encoding = groups[2]
hrp, genesisBytes = decode_bech32_identifier(bech32_encoding)

In [12]:
identifierComponents["hrp"] = hrp
identifierComponents["genesisBytes"] = genesisBytes

In [13]:
identifierComponents

{'version': 1,
 'network': 'regtest',
 'hrp': 'k',
 'genesisBytes': b'\x03n\xac\xa6=\x04\xf3\xa8\xc1\xb7\x01\xd6?`\xe2a~O\xd0\xc4\xaf\xff#\xf1\xd1\xcc\x93*\x1a\x1c\xbe;\x89'}

In [14]:

import copy
serialized_components = copy.deepcopy(identifierComponents)
serialized_components["hexGenesisBytes"] = serialized_components["genesisBytes"].hex()
del serialized_components["genesisBytes"]

In [15]:
serialized_components

{'version': 1,
 'network': 'regtest',
 'hrp': 'k',
 'hexGenesisBytes': '036eaca63d04f3a8c1b701d63f60e2617e4fd0c4afff23f1d1cc932a1a1cbe3b89'}

In [16]:
import json

file_path = f"{folder_path}/identifierComponents.json"
with open(file_path, "w") as f:
    json.dump(serialized_components, f, indent=2)

## Resolve Initial Document

This algorithm specifies how to resolve an initial DID document and validate
it against the `identifier` for a specific **did:btc1**. The algorithm takes as
inputs a **did:btc1** `identifier`, `identifierComponents` object and a
`resolutionsOptions` object. This algorithm returns a valid `initialDocument`
for that identifier.

In [17]:
print(identifierComponents["hrp"])
if identifierComponents["hrp"] == "k":
    print("Deterministically Generate Initial DID Document")
elif identifierComponents["hrp"] == "x":
    raise "Not Implemented"

k
Deterministically Generate Initial DID Document


In [18]:
from libbtc1.did import resolve_deterministic

initial_document = resolve_deterministic(did_to_resolve, identifierComponents)

In [19]:
import json
print(json.dumps(initial_document, indent=2))

{
  "id": "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc",
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://did-btc1/TBD/context"
  ],
  "verificationMethod": [
    {
      "id": "#initialKey",
      "type": "Multikey",
      "controller": "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc",
      "publicKeyMultibase": "zQ3shn68faoXE2EqCTtefQXNLgaTa7ZohG2ftZjgXphStJsGc"
    }
  ],
  "authentication": [
    "#initialKey"
  ],
  "assertionMethod": [
    "#initialKey"
  ],
  "capabilityInvocation": [
    "#initialKey"
  ],
  "capabilityDelegation": [
    "#initialKey"
  ],
  "service": [
    {
      "id": "#initialP2PKH",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:moFJwqLXBDmw4rnWQm9c3ag4kSdFxD5yiz"
    },
    {
      "id": "#initialP2WPKH",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:bcrt1q2n9edlz3yehahctcj6p93lzznhz9m0kzp67ung"
    },
    {
      "id": "#initialP2

In [20]:
file_path = f"{folder_path}/initialDidDocument.json"
with open(file_path, "w") as f:
    json.dump(initial_document, f, indent=2)

## Resolve Target Document

This algorithm resolves a DID document from an initial document by walking the
Bitcoin blockchain to identify Beacon Signals that announce DID Update Payloads
applicable to the **did:btc1** identifier being resolved. The algorithm takes
in an `initialDocument` and a set of `resolutionOptions`. The algorithm returns
a valid `targetDocument` or throws an error.

1. If `resolutionOptions.versionId` is not null, set `targetVersionId` to
   `resolutionOptions.versionId`.
1. Else if `resolutionOptions.versionTime` is not null, set `targetTime` to
   `resolutionOptions.versionTime`.
1. Set `targetBlockheight` to the result of passing `targetTime` to the algorithm
   [Determine Target Blockheight].
1. Set `sidecarData` to `resolutionOptions.sidecarData`.
1. Set `currentVersionId` to 1.
1. If `currentVersionId` equals `targetVersionId` return `initialDocument`.
1. Set `updateHashHistory` to an empty array.
1. Set `contemporaryBlockheight` to 0.
1. Set `contemporaryDIDDocument` to the `initialDocument`.
1. Set `targetDocument` to the result of calling the [Traverse Blockchain History]
   algorithm passing in `contemporaryDIDDocument`, `contemporaryBlockheight`,
   `currentVersionId`, `targetVersionId`, `targetBlockheight`, `updateHashHistory`, 
   and `sidecarData`.
1. Return `targetDocument`.

### 1. If resolutionOptions.versionId is not null, set targetVersionId to resolutionOptions.versionId.
### 2. Else if `resolutionOptions.versionTime` is not null, set `targetTime` to `resolutionOptions.versionTime`.

In [21]:
versionId = resolutionOptions.get("versionId")
versionTime = resolutionOptions.get("versionTime")
targetTime = None
requestVersionId = None
if versionId:
    requestVersionId = versionId
elif versionTime:
    targetTime = versionTime

### 3. Set targetBlockheight to the result of passing targetTime to the algorithm [Determine Target Blockheight].

## Determine Target Blockheight

This algorithm takes in an OPTIONAL Unix `targetTime` and returns a Bitcoin
`blockheight`.

In [22]:
required_confirmations = 6
if targetTime:
    raise "NotImplemented"
else:
    print(f"find latest block with at least {required_confirmations} confimations")



find latest block with at least 6 confimations


In [23]:
from bitcoinrpc import BitcoinRPC
rpc = BitcoinRPC.from_config("http://localhost:18443", ("polaruser", "polarpass"))
await rpc.getconnectioncount()

0

In [24]:
best_blockhash = await rpc.acall("getbestblockhash", {})
bestblock = await rpc.acall("getblock", {"blockhash": best_blockhash})

In [25]:
confirmations = bestblock["confirmations"]
bestblock_height = bestblock["height"]

In [26]:
confirmations

1

In [27]:
targetblockheight = bestblock_height + (confirmations - required_confirmations)

In [28]:
targetblockheight

175

### 4. Set `sidecarData` to `resolutionOptions.sidecarData`.

In [29]:
sidecarData = resolutionOptions["sidecarData"]

### 5. Set `currentVersionId` to 1.


In [30]:
currentVersionId = 1

### 6. If `currentVersionId` equals `targetVersionId` return `initialDocument`.

In [31]:
if currentVersionId == requestVersionId:
    print("returning", initial_document)

### 7. Set `updateHashHistory` to an empty array.

In [32]:
updateHashHistory = []

### 8. Set `contemporaryBlockheight` to 0.

Not sure if this should be 0 or 1.

In [33]:
contemporaryBlockheight = 1

### 9. Set contemporaryDIDDocument to initialDocument

In [34]:
import copy
contemporaryDidDocument = copy.deepcopy(initial_document)

### 10. Set `targetDocument` to the result of calling the [Traverse Blockchain History] algorithm passing in `contemporaryDIDDocument`, `contemporaryBlockheight`,  `currentVersionId`, `targetVersionId`, `targetBlockheight`, `updateHashHistory`, and `sidecarData`.

# Traverse Blockchain History

<a id="traverse_blockchain_history"></a>


### 1. Set `contemporaryHash` to the result of passing `contemporaryDIDDocument` into the [JSON Canonicalization and Hash] algorithm.

In [64]:
import jcs
from buidl.helper import sha256
canonicalDocument = jcs.canonicalize(contemporaryDidDocument)

contemporaryHash = sha256(canonicalDocument)

print(contemporaryHash)

b'\xa4:Xf\xb3g:\x89\xd3LG^\x02\x1b3\x8b\xd0\xec#Gu_8\x87-\x9a\xed\t\xea3\xcfv'


### 2. Find all `beacons` in `contemporaryDIDDocument`: All `service` in `contemporaryDIDDocument.services` where `service.type` equals one of `SingletonBeacon`, `CIDAggregateBeacon` and `SMTAggregateBeacon` Beacon.

In [65]:
import copy
beacons = []

for service in contemporaryDidDocument["service"]:
    serviceType = service["type"]
    if serviceType == "SingletonBeacon" or serviceType == "CIDAggregateBeacon" or serviceType == "SMTAggregateBeacon":
        beacons.append(copy.deepcopy(service))


print(beacons)

[{'id': '#initialP2PKH', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:moFJwqLXBDmw4rnWQm9c3ag4kSdFxD5yiz'}, {'id': '#initialP2WPKH', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:bcrt1q2n9edlz3yehahctcj6p93lzznhz9m0kzp67ung'}, {'id': '#initialP2TR', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:bcrt1p6rs5tnq94rt4uu5edc9luahlkyphk30yk8smwfzurpc8ru06vcws8ylq7l'}]


### 3. For each `beacon` in `beacons` convert the `beacon.serviceEndpoint` to a Bitcoin address following **[BIP21](https://github.com/bitcoin/bips/blob/master/bip-0021.mediawiki)**. Set `beacon.address` to the Bitcoin address.

In [66]:
for beacon in beacons:
    serviceEndpoint = beacon["serviceEndpoint"]
    beacon["address"] = serviceEndpoint.replace("bitcoin:", "")

print(beacons)

[{'id': '#initialP2PKH', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:moFJwqLXBDmw4rnWQm9c3ag4kSdFxD5yiz', 'address': 'moFJwqLXBDmw4rnWQm9c3ag4kSdFxD5yiz'}, {'id': '#initialP2WPKH', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:bcrt1q2n9edlz3yehahctcj6p93lzznhz9m0kzp67ung', 'address': 'bcrt1q2n9edlz3yehahctcj6p93lzznhz9m0kzp67ung'}, {'id': '#initialP2TR', 'type': 'SingletonBeacon', 'serviceEndpoint': 'bitcoin:bcrt1p6rs5tnq94rt4uu5edc9luahlkyphk30yk8smwfzurpc8ru06vcws8ylq7l', 'address': 'bcrt1p6rs5tnq94rt4uu5edc9luahlkyphk30yk8smwfzurpc8ru06vcws8ylq7l'}]


### 4. Set `nextSignals` to the result of calling algorithm Find Next Signals passing in `contemporaryBlockheight` and `beacons`.


# Find Next Signals

This algorithm takes in a `contemporaryBlockheight` and a set of `beacons` and
finds the next Bitcoin block containing Beacon Signals from one or more of the
`beacons`.

In [67]:
from buidl.tx import Tx, TxIn

async def find_next_signals(contemporaryBlockheight, targetBlockheight, beacons):
    # 1. Get Bitcoin block at contemporaryBlockheight
    blockhash = await rpc.acall("getblockhash", {"height": contemporaryBlockheight})
    block = await rpc.acall("getblock", {"blockhash": blockhash})
    
    # 2. Set `beaconSignals` to an empty array.
    beaconSignals = {
        "signals": [],
        "blockheight": contemporaryBlockheight
    }
    
    # 3.For each `txid` in `block.tx`: 
    # check to see if any transaction inputs are spends from one of the Beacon addresses.
    # If they are, create a `signal` object containing the following fields and push
    # `signal` to `beaconSignals`:
    for txid in block["tx"]:
        # print(txid)
        #  Skip coinbase and genesis coinbase 4a5...
        if txid != "4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b" or txid != "0000000000000000000000000000000000000000000000000000000000000000":
            # 1. Fetch transaction for txid
            tx_hex = await rpc.acall("getrawtransaction", {"txid": txid})
            tx = Tx.parse_hex(tx_hex)
            
            
            # For each tx input in transaction
            for tx_in in tx.tx_ins:    
                prev_txid = tx_in.prev_tx.hex()
                # If tx_in is NOT a coinbase transaction. Coinbase can never be a beacon signal
                if prev_txid != "0000000000000000000000000000000000000000000000000000000000000000":
                    # Fetch previous tx
                    prev_tx_hex = await rpc.acall("getrawtransaction", {"txid": prev_txid})
                    prev_tx = Tx.parse_hex(prev_tx_hex)
                    # Get spent output
                    spent_tx_output = prev_tx.tx_outs[tx_in.prev_index]
                    print("Bitcoin block at height : ", contemporaryBlockheight)
                    print(spent_tx_output.script_pubkey.address(network="regtest"))
                    # Get address for the spent output 
                    spent_tx_output_address = spent_tx_output.script_pubkey.address(network="regtest")
                    from_beacon = None
                    # Check if address is a beacon address
                    for beacon in beacons:
                        if beacon["address"] == spent_tx_output_address:
                            from_beacon = {
                                "beaconId": beacon["id"],
                                "beaconType": beacon["type"],
                                "tx": tx
                            }
                            beaconSignals["signals"].append(from_beacon)
                            print("Found Beacon Signal", tx)
                            break
                    # If found beacon, no need to check other inputs
                    # Is that true?
                    if from_beacon:
                        break

    if contemporaryBlockheight == targetBlockheight:
        return beaconSignals

    
    if len(beaconSignals["signals"]) == 0:
        next_blockheight = contemporaryBlockheight + 1
        beaconSignals = await find_next_signals(next_blockheight, targetBlockheight, beacons)

    return beaconSignals
                    


In [68]:

nextSignals = await find_next_signals(contemporaryBlockheight, targetblockheight, beacons)

In [69]:
serializedSignals = copy.deepcopy(nextSignals)

In [70]:
serializedSignals

{'signals': [], 'blockheight': 175}

In [71]:
serializedSignals["signals"][0]["txhex"] = serializedSignals["signals"][0]["tx"].serialize().hex()
del serializedSignals["signals"][0]["tx"]

IndexError: list index out of range

In [72]:
serializedSignals

{'signals': [], 'blockheight': 175}

In [44]:
block_path = f"{folder_path}/block{serializedSignals['blockheight']}"

In [45]:
# Ensure the folder exists
if not os.path.exists(block_path):
    os.makedirs(block_path)

In [46]:
file_path = f"{block_path}/nextSignals.json"
with open(file_path, "w") as f:
    json.dump(serializedSignals, f, indent=2)

## 5. Set `contemporaryBlockheight` to `nextSignals.contemporaryBlockheight`.


In [75]:
contemporaryBlockheight = nextSignals["blockheight"]

### 5. Set `signals` to `nextSignals.signals`.


In [48]:
signals = nextSignals["signals"]

### 6. Set `updates` to the result of calling algorithm [Process Beacon Signals] passing in `signals` and `sidecarData`.

In [49]:
signals

[{'beaconId': '#initialP2WPKH',
  'beaconType': 'SingletonBeacon',
  'tx': 
  tx: 5f8cd13f39fa509b1cdfdc7c6588b6cda99e82202e9498dff9f37dc99d4a1e10
  version: 1
  locktime: 0
  tx_ins:
  a4646b7d0d429a7f4c73d9e086a5b74c147e64219751a334c2767abdea7c5a89:1
  tx_outs:
  0:OP_RETURN 335a39b28577022748ff4ce580a3a9e17d8824d6bb73fd01b6701e1564a8e01a 
  19999650:OP_0 54cb96fc51266fdbe178968258fc429dc45dbec2 }]

# Process Beacon Signals

This algorithm takes in an array of struct `beaconSignals` and attempts
to process these signals according the type of the Beacon they were produced by.
Each `beaconSignal` struct contains the properties `beaconId`, `beaconType`, and
a `tx`. Additionally, this algorithm takes in `sidecarData` passed into the
resolver through the `resolutionOptions`. If `sidecarData` is present it is used
to process the Beacon Signals.

In [50]:
print(signals[0]["tx"])
print(json.dumps(sidecarData, indent=2))


tx: 5f8cd13f39fa509b1cdfdc7c6588b6cda99e82202e9498dff9f37dc99d4a1e10
version: 1
locktime: 0
tx_ins:
a4646b7d0d429a7f4c73d9e086a5b74c147e64219751a334c2767abdea7c5a89:1
tx_outs:
0:OP_RETURN 335a39b28577022748ff4ce580a3a9e17d8824d6bb73fd01b6701e1564a8e01a 
19999650:OP_0 54cb96fc51266fdbe178968258fc429dc45dbec2 

{
  "did": "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc",
  "signalsMetadata": {
    "5f8cd13f39fa509b1cdfdc7c6588b6cda99e82202e9498dff9f37dc99d4a1e10": {
      "updatePayload": {
        "@context": [
          "https://w3id.org/security/v2",
          "https://w3id.org/zcap/v1",
          "https://w3id.org/json-ld-patch/v1"
        ],
        "patch": [
          {
            "op": "add",
            "path": "/service/3",
            "value": {
              "id": "#linked-domain",
              "type": "LinkedDomains",
              "serviceEndpoint": "https://contact-me.com"
            }
          }
        ],
        "sourceHash": "9kSA9j

In [51]:
def process_beacon_signals(beaconSignals, sidecarData):
    # 1. Set `updates` to an empty array.
    updates = []
    # 2. For `beaconSignal` in `beaconSignals`:
    for beaconSignal in beaconSignals:
        beaconType = beaconSignal["beaconType"]
        signalTx = beaconSignal["tx"]
        signalId = signalTx.id()
        signalSidecarData = sidecarData["signalsMetadata"].get(signalId)
        didUpdatePayload = None
        if beaconType == "SingletonBeacon":
            didUpdatePayload = process_singleton_beacon_signal(signalTx, signalSidecarData)
        # Handle other cases
        else:
            raise "NotImplemented"

        if didUpdatePayload:
            updates.append(didUpdatePayload)
    return updates

# Process Singleton Beacon Signal

This algorithm is called by the [Process Beacon Signals] algorithm as part of the
[Read] operation. It takes as inputs a Bitcoin transaction, `tx`, representing a ::Beacon Signal::
and a optional object, `signalSidecarData`, containing any sidecar data provided to the 
resolver for the ::Beacon Signal:: identified by the Bitcoin transaction identifier.

The algorithm returns the ::DID Update payload:: announced by the ::Beacon Signal:: or throws
an error.

1. Initialize a `txOut` variable to the 0th transaction output of the `tx`.
1. Set `didUpdatePayload` to null.
1. Check `txOut` is of the format `[OP_RETURN, OP_PUSH32, <32bytes>]`, if not,
   then return `didUpdatePayload`. The Bitcoin transaction is not a ::Beacon Signal::.
1. Set `hashBytes` to the 32 bytes in the `txOut`.
1. If `signalSidecarData`:
   1. Set `didUpdatePayload` to `signalSidecarData.updatePayload`
   1. Set `updateHashBytes` to the result of passing `didUpdatePayload` to the 
      [JSON Canonicalization and Hash] algorithm.
   1. If `updateHashBytes` does not equal `hashBytes`, MUST throw an `invalidSidecarData` error.
   1. Return `didUpdatePayload`
1. Else:
   1. Set `didUpdatePayload` to the result of passing `hashBytes` into the 
      [Fetch Content from Addressable Storage] algorithm.
   1. If `didUpdatePayload` is null, MUST raise a `latePublishingError`. May identify Beacon Signal
      to resolver and request additional ::Sidecar data:: be provided.
1. Return `didUpdatePayload`.

In [52]:
from ipfs_cid import cid_sha256_wrap_digest


def process_singleton_beacon_signal(tx, signalSidecarData):
    
    txOut = tx.tx_outs[0]
    didUpdatePayload = None
    # Note: No OP_PUSH32. It is implied by the length of the bytes.
    if (txOut.script_pubkey.commands[0] != 106 and len(txOut.script_pubkey.commands[1]) != 32):
        print("Not a beacon signal")
        return didUpdatePayload
    hashBytes = txOut.script_pubkey.commands[1]
    if signalSidecarData:
        didUpdatePayload = signalSidecarData["updatePayload"]
        updateHashBytes = sha256(jcs.canonicalize(didUpdatePayload))
        

        if updateHashBytes != hashBytes:
            raise Exception("Invalid Sidecar Data")
        print("DID Update verified", json.dumps(didUpdatePayload, indent=2))
        return didUpdatePayload
    else:
        payload_cid = cid_sha256_wrap_digest(hashBytes)
        print("TODO: Fetch CID from CAS", payload_cid)
        
        

In [53]:
updates = process_beacon_signals(signals, sidecarData)

DID Update verified {
  "@context": [
    "https://w3id.org/security/v2",
    "https://w3id.org/zcap/v1",
    "https://w3id.org/json-ld-patch/v1"
  ],
  "patch": [
    {
      "op": "add",
      "path": "/service/3",
      "value": {
        "id": "#linked-domain",
        "type": "LinkedDomains",
        "serviceEndpoint": "https://contact-me.com"
      }
    }
  ],
  "sourceHash": "9kSA9j3z2X3a26yAdJi6nwg31qyfaHMCU1u81ZrkHirM",
  "targetHash": "C45TsdfkLZh5zL6pFfRmK93X4EdHusbCDwvt8d7Xs3dP",
  "targetVersionId": 2,
  "proof": {
    "type": "DataIntegrityProof",
    "cryptosuite": "bip340-jcs-2025",
    "verificationMethod": "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc#initialKey",
    "proofPurpose": "capabilityInvocation",
    "capability": "urn:zcap:root:did%3Abtc1%3Aregtest%3Ak1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc",
    "capabilityAction": "Write",
    "@context": [
      "https://w3id.org/security/v2",
      "https://w3id.o

In [54]:
updates

[{'@context': ['https://w3id.org/security/v2',
   'https://w3id.org/zcap/v1',
   'https://w3id.org/json-ld-patch/v1'],
  'patch': [{'op': 'add',
    'path': '/service/3',
    'value': {'id': '#linked-domain',
     'type': 'LinkedDomains',
     'serviceEndpoint': 'https://contact-me.com'}}],
  'sourceHash': '9kSA9j3z2X3a26yAdJi6nwg31qyfaHMCU1u81ZrkHirM',
  'targetHash': 'C45TsdfkLZh5zL6pFfRmK93X4EdHusbCDwvt8d7Xs3dP',
  'targetVersionId': 2,
  'proof': {'type': 'DataIntegrityProof',
   'cryptosuite': 'bip340-jcs-2025',
   'verificationMethod': 'did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc#initialKey',
   'proofPurpose': 'capabilityInvocation',
   'capability': 'urn:zcap:root:did%3Abtc1%3Aregtest%3Ak1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc',
   'capabilityAction': 'Write',
   '@context': ['https://w3id.org/security/v2',
    'https://w3id.org/zcap/v1',
    'https://w3id.org/json-ld-patch/v1'],
   'proofValue': 'z3yfzVGdoDF4s8y4Bk8JeV9Xu

In [55]:
file_path = f"{block_path}/updates.json"
with open(file_path, "w") as f:
    json.dump(updates, f, indent=2)

### 7. Set `orderedUpdates` to the list of `updates` ordered by the `targetVersionId` property.

In [56]:
updates.sort(key=lambda update: update["targetVersionId"])

### 8. For `update` in `orderedUpdates`:
1. If `update.targetVersionId` is less than or equal to `currentVersionId`,
   run Algorithm [Confirm Duplicate Update] passing in `update`,
   `documentHistory`, and `contemporaryHash`.
1. If `update.targetVersionId` equals `currentVersionId + 1`:
    1.  Check that `update.sourceHash` equals `contemporaryHash`, else MUST
        raise LatePublishing error.
    1.  Set `contemporaryDIDDocument` to the result of calling [Apply DID Update]
        algorithm passing in `contemporaryDIDDocument`, `update`.
    1.  Increment `currentVersionId`
    1.  If `currentVersionId` equals `targetVersionId` return
        `contemporaryDIDDocument`.
    1.  Set `updateHash` to the sha256 hash of the `update`.
    1.  Push `updateHash` onto `updateHashHistory`.
    1.  Set `contemporaryHash` to the SHA256 hash of the
        `contemporaryDIDDocument`.
1.  If `update.targetVersionId` is greater than `currentVersionId + 1`, MUST
    throw a LatePublishing error.

# Confirm Duplicate Update

This algorithm takes in a DID Update Payload and verifies that the update is a
duplicate against the hash history of previously applied updates.
The algorithm takes in an `update` and an array of hashes, `updateHashHistory`.
It throws an error if the `update` is not a duplicate, otherwise it returns.
TODO: does this algorithm need  `contemporaryHash` passed in?

In [57]:
def confirm_duplicate_update(update, updateHashHistory):

    updateHash = sha256(jcs.canonicalize(update))
    # Note: version starts at 1, index starts at 0
    updateHashIndex = update["targetVersionId"] - 2
    historicalUpdateHash = updateHashHistory[updateHashIndex]
    if (historicalUpdateHash != updateHash):
        raise Exception("Late Publishing Error")
    return    

# Apply DID Update

This algorithm attempts to apply a DID Update to a DID document, it first
verifies the proof on the update is a valid capabilityInvocation of the root
authority over the DID being resolved. Then it applies the JSON patch
transformation to the DID document, checks the transformed DID document
matches the targetHash specified by the update and validates it is a conformant
DID document before returning it. This algorithm takes inputs
`contemporaryDIDDocument` and an `update`.

1. Instantiate a `schnorr-secp256k1-2025` `cryptosuite` instance.
1. Set `expectedProofPurpose` to `capabilityInvocation`.
1. Set `mediaType` to ???? TODO
1. Set `documentBytes` to the bytes representation of `update`.
1. Set `verificationResult` to the result of passing `mediaType`, `documentBytes`,
   `cryptosuite`, and `expectedProofPurpose` into the
   [Verify Proof algorithm](https://w3c.github.io/vc-data-integrity/#verify-proof)
   defined in the VC Data Integrity specification.
1. TODO: HOW DO WE ENSURE THAT THE PROOF IS A VALID INVOCATION OF THE ROOT
   CAPABILITY derived using [Derive Root Capability from **did:btc1** Identifier]
   algorithm
1. Set `targetDIDDocument` to a copy of `contemporaryDIDDocument`.
1. Use JSON Patch to apply the `update.patch` to the `targetDIDDOcument`.
1. Verify that `targetDIDDocument` is conformant with the data model specified
   by the DID Core specification.
1. Set `targetHash` to the SHA256 hash of `targetDIDDocument`.
1. Check that `targetHash` equals `update.targetHash`, else raise InvalidDIDUpdate
   error.
1. Return `targetDIDDocument`.

In [58]:
from di_bip340.multikey import SchnorrSecp256k1Multikey
from di_bip340.data_integrity_proof import DataIntegrityProof
from di_bip340.cryptosuite import Bip340JcsCryptoSuite
from libbtc1.verificationMethod import get_key_for_verification_method
import urllib
import jsonpatch

def dereference_root_capability(capability_id):
    
    components = capability_id.split(":")
    assert(len(components) == 4)
    assert(components[0] == "urn")
    assert(components[1] == "zcap")
    assert(components[2] == "root")
    uri_encoded_id = components[3]
    btc1Identifier = urllib.parse.unquote(uri_encoded_id)
    root_capability = {
        "@context": "https://w3id.org/zcap/v1",
        "id": capability_id,
        "controller": btc1Identifier,
        "invocationTarget": btc1Identifier
    }
    return root_capability


def apply_did_update(contemporaryDIDDocument, update):
    # Retrieve the verification method used to secure the proof from the contemporary DID document
    capability_id = update["proof"]["capability"]

    root_capability = dereference_root_capability(capability_id)
    
    proof_vm_id = update["proof"]["verificationMethod"]
    btc1_identifier = contemporaryDIDDocument["id"]
    verification_method = None
    for vm in contemporaryDIDDocument["verificationMethod"]:
        vm_id = vm["id"]
        if vm_id[0] == "#":
            vm_id = f"{btc1_identifier}{vm_id}"
        if vm_id == proof_vm_id:
            print("Verification Method found", vm)
            verification_method = vm
    if verification_method == None:
        raise Exception("Invalid Proof on Update Payload")
    print(json.dumps(verification_method, indent=2))
    multikey = SchnorrSecp256k1Multikey.from_verification_method(verification_method)

    # Instantiate a schnorr-secp256k1-2025 cryptosuite instance.
    cryptosuite = Bip340JcsCryptoSuite(multikey)
    di_proof = DataIntegrityProof(cryptosuite=cryptosuite)

    mediaType = "application/json"

    expected_proof_purpose = "capabilityInvocation"

    update_bytes = json.dumps(update)

    verificationResult = di_proof.verify_proof(mediaType, update_bytes, expected_proof_purpose, None, None)

    if not verificationResult["verified"]:
        raise Exception("invalidUpdateProof")

    target_did_document = copy.deepcopy(contemporaryDIDDocument)

    
    update_patch = update["patch"]


    patch = jsonpatch.JsonPatch(update_patch)
    
    target_did_document = patch.apply(target_did_document)

    target_hash = sha256(jcs.canonicalize(target_did_document))

    update_target_hash = base58.b58decode(update["targetHash"])
    assert(target_hash == update_target_hash)

    return target_did_document

    

In [59]:
import base58
for update in updates:
    print(update)
    targetVersionId = update["targetVersionId"]
    if targetVersionId <= currentVersionId:
        confirm_duplicate_update(update, updateHashHistory)
    if targetVersionId == currentVersionId + 1:
        if base58.b58decode(update["sourceHash"]) != contemporaryHash:
            raise Exception("Late Publishing")
        contemporaryDidDocument = apply_did_update(contemporaryDidDocument, update)
        currentVersionId +=1
        if currentVersionId == targetVersionId:
            print("Found document for target version", contemporaryDidDocument)
        updateHash = sha256(jcs.canonicalize(update))
        updateHashHistory.append(updateHash)
        contemporaryHash = sha256(jcs.canonicalize(contemporaryDidDocument))
    if targetVersionId > currentVersionId + 1:
        raise Exception("Late publishing")
        

{'@context': ['https://w3id.org/security/v2', 'https://w3id.org/zcap/v1', 'https://w3id.org/json-ld-patch/v1'], 'patch': [{'op': 'add', 'path': '/service/3', 'value': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}], 'sourceHash': '9kSA9j3z2X3a26yAdJi6nwg31qyfaHMCU1u81ZrkHirM', 'targetHash': 'C45TsdfkLZh5zL6pFfRmK93X4EdHusbCDwvt8d7Xs3dP', 'targetVersionId': 2, 'proof': {'type': 'DataIntegrityProof', 'cryptosuite': 'bip340-jcs-2025', 'verificationMethod': 'did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc#initialKey', 'proofPurpose': 'capabilityInvocation', 'capability': 'urn:zcap:root:did%3Abtc1%3Aregtest%3Ak1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc', 'capabilityAction': 'Write', '@context': ['https://w3id.org/security/v2', 'https://w3id.org/zcap/v1', 'https://w3id.org/json-ld-patch/v1'], 'proofValue': 'z3yfzVGdoDF4s8y4Bk8JeV9XuZw1nMeMtNW3x5brEm7DNtmWZkNBPbCLzUBJRpctBj9QJL1dydm94ZNsPxosPnkPP

In [60]:
contemporaryDidDocument

{'id': 'did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc',
 '@context': ['https://www.w3.org/ns/did/v1', 'https://did-btc1/TBD/context'],
 'verificationMethod': [{'id': '#initialKey',
   'type': 'Multikey',
   'controller': 'did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc',
   'publicKeyMultibase': 'zQ3shn68faoXE2EqCTtefQXNLgaTa7ZohG2ftZjgXphStJsGc'}],
 'authentication': ['#initialKey'],
 'assertionMethod': ['#initialKey'],
 'capabilityInvocation': ['#initialKey'],
 'capabilityDelegation': ['#initialKey'],
 'service': [{'id': '#initialP2PKH',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:moFJwqLXBDmw4rnWQm9c3ag4kSdFxD5yiz'},
  {'id': '#initialP2WPKH',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:bcrt1q2n9edlz3yehahctcj6p93lzznhz9m0kzp67ung'},
  {'id': '#initialP2TR',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:bcrt1p6rs5tnq94rt4uu5edc9luahlkyphk30yk8smwfzurpc8ru06vcws8ylq7l'},
 

In [61]:
file_path = f"{block_path}/contemporaryDidDocument.json"
with open(file_path, "w") as f:
    json.dump(contemporaryDidDocument, f, indent=2)

## 10. If `contemporaryBlockheight` equals `targetBlockheight`, return `contemporaryDIDDocument`


In [76]:
if contemporaryBlockheight == targetblockheight:
    print("Resolution Complete")
    print(json.dumps(contemporaryDidDocument, indent=2))
    file_path = f"{folder_path}/targetDidDocument.json"
    with open(file_path, "w") as f:
        json.dump(contemporaryDidDocument, f, indent=2)

Resolution Complete
{
  "id": "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc",
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://did-btc1/TBD/context"
  ],
  "verificationMethod": [
    {
      "id": "#initialKey",
      "type": "Multikey",
      "controller": "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc",
      "publicKeyMultibase": "zQ3shn68faoXE2EqCTtefQXNLgaTa7ZohG2ftZjgXphStJsGc"
    }
  ],
  "authentication": [
    "#initialKey"
  ],
  "assertionMethod": [
    "#initialKey"
  ],
  "capabilityInvocation": [
    "#initialKey"
  ],
  "capabilityDelegation": [
    "#initialKey"
  ],
  "service": [
    {
      "id": "#initialP2PKH",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:moFJwqLXBDmw4rnWQm9c3ag4kSdFxD5yiz"
    },
    {
      "id": "#initialP2WPKH",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:bcrt1q2n9edlz3yehahctcj6p93lzznhz9m0kzp67ung"
    },
    {
   

In [74]:
contemporaryBlockheight

169

## 11. Increment contemporaryBlockheight

In [63]:
contemporaryBlockheight+= 1

## 12. Set `targetDocument` to the result of calling the [Traverse Blockchain History] algorithm passing in `contemporaryDIDDocument`, `contemporaryBlockheight`, `currentVersionId`, `targetVersionId`, `targetBlockheight`, `documentHistory`, and `sidecarData`.

This recursively calls itself